<a href="https://colab.research.google.com/github/JeanPorta/Deteccion-automatizada-de-textos-MPU/blob/main/detecci%C3%B3n_txt_generado_ia_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers scikit-learn

# Libreria

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_cosine_schedule_with_warmup
from sklearn.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm
import random
from torch.cuda.amp import autocast

# Carga de DataSet

In [ ]:
train_df = pd.read_csv('/content/en_train_cleaned.csv')
test_df = pd.read_csv('/content/en_test_cleaned.csv')

texts = train_df['answer'].tolist()
labels = train_df['label'].tolist()
val_texts = test_df['answer'].tolist()
val_labels = test_df['label'].tolist()

# DataSet con Text Multiscaling

In [ ]:
class TextMultiscaleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128, psent=0.25):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.psent = psent

    def __len__(self):
        return len(self.labels)

    def multiscale_text(self, text):
        sentences = text.split('.')
        new_sentences = [s for s in sentences if random.random() > self.psent]
        if len(new_sentences) == 0:
            new_sentences = [random.choice(sentences)]
        return '.'.join(new_sentences)

    def __getitem__(self, idx):
        text = self.multiscale_text(self.texts[idx])
        enc = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        length_value = min(len(text.split()), 512)
        item['length'] = torch.tensor(length_value, dtype=torch.long)
        return item

# MPU Loss

In [ ]:
def mpu_loss(logits, labels, lengths, gamma=0.4, p_token=0.2):
    ce_loss = nn.CrossEntropyLoss()(logits, labels)
    prior = 2 * p_token * (1 - torch.exp(-lengths.float() / 50)).to(logits.device)
    probs = torch.softmax(logits, dim=1)[:, 1]
    pu = torch.mean(prior * (1 - probs) + (1 - prior) * probs)
    return ce_loss + gamma * pu

# Entrenamiento

In [ ]:
def train_mpu_roberta(texts, labels, val_texts, val_labels, epochs=5, batch_size=64, lr=2e-5, max_length=128):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)

    dataset = TextMultiscaleDataset(texts, labels, tokenizer, max_length, psent=0.25)
    val_dataset = TextMultiscaleDataset(val_texts, val_labels, tokenizer, max_length, psent=0)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.1*len(loader)*epochs, num_training_steps=len(loader)*epochs)

    scaler = torch.cuda.amp.GradScaler()
    best_f1 = 0

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for batch in tqdm(loader, desc=f"Epoch {epoch+1}"):
            optimizer.zero_grad()
            #with torch.cuda.amp.autocast():
            with autocast():
                outputs = model(input_ids=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
                loss = mpu_loss(outputs.logits, batch['labels'].to(device), batch['length'].to(device), gamma=0.4, p_token=0.2)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            epoch_loss += loss.item()

        # Validación
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for batch in val_loader:
                outputs = model(input_ids=batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
                pred_labels = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                preds.extend(pred_labels)
                trues.extend(batch['labels'].numpy())

        f1 = f1_score(trues, preds)
        prec = precision_score(trues, preds)
        rec = recall_score(trues, preds)

        print(f"\nEpoch {epoch+1}: Loss={epoch_loss/len(loader):.4f} | F1={f1:.4f} | Precision={prec:.4f} | Recall={rec:.4f}")

        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), 'best_mpu_roberta.pt')
            print("Mejor modelo guardado.")

    print(f"Entrenamiento completado. Mejor F1 alcanzado: {best_f1:.4f}")

# Ejecución

In [ ]:
train_mpu_roberta(texts, labels, val_texts, val_labels, epochs=5, batch_size=64, lr=2e-5, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...


Epoch 3: Loss=0.2886 | F1=0.9160 | Precision=0.8546 | Recall=0.9870


Epoch 4:   0%|          | 0/5821 [00:00<?, ?it/s]/tmp/ipython-input-3-3564039057.py:82: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 4: 100%|██████████| 5821/5821 [07:49<00:00, 12.40it/s]



Epoch 4: Loss=0.2718 | F1=0.9180 | Precision=0.8562 | Recall=0.9895


Epoch 5:   0%|          | 0/5821 [00:00<?, ?it/s]/tmp/ipython-input-3-3564039057.py:82: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 5: 100%|██████████| 5821/5821 [07:48<00:00, 12.42it/s]



Epoch 5: Loss=0.2642 | F1=0.9239 | Precision=0.8673 | Recall=0.9884
Mejor modelo guardado.
Entrenamiento completado. Mejor F1 alcanzado: 0.9239


In [ ]:
!nvidia-smi

Tue Jul  8 20:04:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   52C    P0             83W /  400W |    5777MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----